# Minimal notebook with fetching wind data from HSDS + spatial & vertical interpolation + ANL's obstactle model

### Important HSDS info:
- accessing wind data stored in AWS using HSDS **works only from NREL networks**
- to enable HSDS connectons, you should have a file `~/.hscfg` with the following contents:
```
hs_endpoint = <endpoint>
hs_username = <user>
hs_password = <password>
hs_api_key = <key>
hs_bucket = nrel-pds-hsds
```
- collaborators from outside of NREL should use another notebook with no HSDS operations (i.e. reading pre-saved wind data)

In [2]:
import numpy as np
import pandas as pd
import time
import h5pyd
from dw_tap.data_fetching import getData
from dw_tap.process_geojson import get_candidate
from dw_tap.process_geojson import get_obstacles
from dw_tap.power_output import estimate_power_output
from dw_tap.lom import run_lom

z_turbine = 40 # turbine height in [m]
lat, lon = 39.32856, -89.40238
obstacle_file = "../bergey/180-Manual1.geojson"

t_start = time.time()

# Open the wind data "file"
# server endpoint, username, password are found in ~/.hscfg
f = h5pyd.File("/nrel/wtk-us.h5", 'r', bucket="nrel-pds-hsds") 

t_fetch_start = time.time()
# The last three arguments limit data fetching to every 12 hours for 1 year
ws, theta, dates, temp, pres = getData(f, lat, lon, z_turbine, "IDW", True,
                                       start_time_idx=0, end_time_idx=8760, time_stride=12)
t_fetch = time.time() - t_fetch_start
print('Data fetching time: %.2f (s)' % t_fetch)

df_places = get_obstacles("../bergey/180-Manual1.geojson")
x1_turbine,y1_turbine,minx,maxx,miny,maxy = get_candidate(lat, lon)
xy_turbine = [np.array([x1_turbine, y1_turbine])]

t_lom_start = time.time()
predictions_df = \
    run_lom(dates, ws, theta, df_places, xy_turbine, z_turbine, \
            minx, maxx, miny, maxy)
t_lom = time.time() - t_lom_start
print('Running LOM: %.2f (s)' % t_lom)

t_power_start = time.time()
kw, above_curve, below_curve, above_curve_list, below_curve_list = \
    estimate_power_output(predictions_df, temp, pres)  
t_power = time.time() - t_power_start
print('Power estimation: %.2f (s)' % t_power)

t_total = time.time()-t_start
print('Total time: %.2f (s)' % t_total)

[1763  858]
859
[ 96.79590445 502.03465961]
Data fetching time: 70.55 (s)


2022-10-11 09:34:13.037644: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


LOM time : 0.05  min
Running LOM: 3.25 (s)
Power estimation: 0.02 (s)
Total time: 74.27 (s)
